In [12]:
import os, json
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util as bsonju  # preserves {"$date": ...}

# READ-ONLY: do not create or modify any document.

def get_latest_empty_form():
    load_dotenv()
    uri = os.getenv("MONGODB_URI")
    database_name = os.getenv("MONGODB_DATABASE")
    collection_name = "sales_invoices"

    if not uri or not database_name:
        print(json.dumps({
            "ok": False,
            "reason": "config_error",
            "message": "Missing MONGODB_URI or MONGODB_DATABASE"
        }, indent=2))
        return

    try:
        client = MongoClient(uri, serverSelectionTimeoutMS=5000)
        db = client[database_name]
        col = db[collection_name]

        # Get the most recent empty form (regardless of how many exist)
        doc = col.find_one({"is_empty": True}, sort=[("created", -1)])

        if not doc:
            out = {
                "ok": False,
                "reason": "none_found",
                "message": "No document with is_empty=True found."
            }
        else:
            out = {
                "ok": True,
                "reason": "latest_found",
                "message": "Latest empty form found.",
                "document": doc
            }

        print(bsonju.dumps(out, indent=2, ensure_ascii=False))

    except Exception as e:
        print(json.dumps({
            "ok": False,
            "reason": "runtime_error",
            "message": str(e)
        }, indent=2))

# Run
get_latest_empty_form()


{
  "ok": true,
  "reason": "latest_found",
  "message": "Latest empty form found.",
  "document": {
    "_id": {
      "$oid": "68f8366d800ddf5dffd44541"
    },
    "id": "1d020019-2c43-4bd8-8a52-caac4aeb9251",
    "previous_form_qr_code": "20251012",
    "current_form_qr_code": "52741246",
    "tin": "123-456-789",
    "location": "JEF Gas Station – Sikatuna Branch",
    "created": {
      "$date": 1761097322765
    },
    "date": {
      "$date": 1761091200000
    },
    "items": [],
    "employees": [
      {
        "role": "cashier",
        "employee_number": "12345",
        "name": "John Smith"
      },
      {
        "role": "recorder",
        "employee_number": "12345",
        "name": null
      }
    ],
    "is_empty": true
  }
}
